# **Global Constants**

In [0]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/gdrive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/My Drive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Teaching/2019-20-BDC/datasets"
DATASET_NODES_URL = "https://github.com/gtolomei/big-data-computing/raw/master/datasets/web-pages.csv.bz2"
DATASET_LINKS_URL = "https://github.com/gtolomei/big-data-computing/raw/master/datasets/web-links.csv.bz2"
GDRIVE_DATASET_NODES_FILE = GDRIVE_DATA_DIR + "/" + DATASET_NODES_URL.split("/")[-1]
GDRIVE_DATASET_LINKS_FILE = GDRIVE_DATA_DIR + "/" + DATASET_LINKS_URL.split("/")[-1]

RANDOM_SEED = 42 # for reproducibility

# **Spark + Google Colab Setup**

## **1.** Install PySpark and related dependencies

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME

## **2.** Import useful Python packages

In [0]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

## **3.** Create Spark context

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050").set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

##**3.a** Add `GraphFrames` to `SparkContext`

[`GraphFrames`](https://graphframes.github.io/graphframes/docs/_site/index.html) is a Python wrapper for [Spark's GraphX API](https://spark.apache.org/graphx/), which unfortunately does not natively support Python at the moment. In order to make `GraphFrames` work, we need to firstly download it and then add it to our current `SparkContext`. The two cells below do exactly that job.

###Download `GraphFrames`
`GraphFrames` can be download from a dedicated section of the [Spark packages website](https://spark-packages.org/package/graphframes/graphframes). Please, check out the latest version available, which is also compliant with the Apache Spark version you are working on (i.e., in our case `spark-2.4.x`), for instance: `0.8.0-spark2.4-s_2.11`. 

In [0]:
GRAPHFRAMES_VERSION="0.8.0-spark2.4-s_2.11" # change this string to the correct/latest version if needed
GRAPHFRAMES_JAR="{:s}.jar".format(GRAPHFRAMES_VERSION)
GRAPHFRAMES_URL="http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/{:s}/graphframes-{:s}".format(GRAPHFRAMES_VERSION, GRAPHFRAMES_JAR)
GRAPHFRAMES_DEST="/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-{:s}".format(GRAPHFRAMES_JAR)

In [0]:
!echo "!curl -L -o \"{GRAPHFRAMES_DEST}\" {GRAPHFRAMES_URL}"
!curl -L -o "{GRAPHFRAMES_DEST}" {GRAPHFRAMES_URL} #!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.0-spark2.4-s_2.11.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.0-spark2.4-s_2.11/graphframes-0.8.0-spark2.4-s_2.11.jar

In [0]:
sc.addPyFile("/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-{:s}".format(GRAPHFRAMES_JAR))

###Import all packages from `graphframes`

In [0]:
from graphframes import *

## **4.** Create <code>ngrok</code> tunnel to check the Spark UI

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## **5.** Link Colab to our Google Drive

In [0]:
# Point Colaboratory to our Google Drive

from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

## **6.** Check everything is ok

In [0]:
spark

In [0]:
sc._conf.getAll()

# **Google Web Graph**

In this notebook, we will be using a dataset from the [SNAP Group](https://snap.stanford.edu/data/web-Google.html)*, which was released in 2002 by Google as a part of [Google Programming Contest](http://www.google.com/programming-contest/). 

More specifically, this dataset contains a snapshot of the **Google Web Graph** consisting of **875,713** nodes (i.e., web pages) connected by **5,105,039** edges (i.e., hyperlinks).

The original dataset comes as a single file; in order to facilitate the usage of `GraphFrames` API, it has been split into **2 sources**: 
-  `web-pages.csv.bz2` containing only the nodes of the graph;
-  `web-links.csv.bz2` containing the links between nodes.

The reason for that is because `GraphFrames` can create a graph from **2 PySpark DataFrame objects**, representing the set of vertices and edges, respectively. There is a naming convention which those two dataframes must be compliant with: the former should contain at least a column named `id` (indicating the node identifiers), whilst the latter should contain at least two columns named `src` and `dst` to denote the identifiers of the *source* and *destination* nodes connected by a (directed) edge, respectively.

For a deeper understanding of the `GraphFrames` package, please refer to the [online user guide](https://graphframes.github.io/graphframes/docs/_site/user-guide.html).

[**SNAP is the acronym for Stanford Network Analysis Project led by Prof. Jure Leskovec*]

# **1. Data Collection**

This is the first step we need to accomplish before going any further. The dataset will be downloaded directly to our Google Drive, as usual.

### **Download dataset file from URL directly to our Google Drive**

In [0]:
def get_data(dataset_url, dest, chunk_size=1024):
  response = requests.get(dataset_url, stream=True)
  if response.status_code == 200:
    with open(dest, "wb") as file:
      for block in response.iter_content(chunk_size=chunk_size): 
        if block: 
          file.write(block)

###**Retrieve the dataset containing nodes (i.e., web pages)** **bold text**

In [0]:
print("Retrieving dataset from URL: {} ...".format(DATASET_NODES_URL))
get_data(DATASET_NODES_URL, GDRIVE_DATASET_NODES_FILE)
print("Dataset successfully retrieved and stored at: {}".format(GDRIVE_DATASET_NODES_FILE))

###**Retrieve the dataset containing edges (i.e., hyperlinks)**

In [0]:
print("Retrieving dataset from URL: {} ...".format(DATASET_LINKS_URL))
get_data(DATASET_LINKS_URL, GDRIVE_DATASET_LINKS_FILE)
print("Dataset successfully retrieved and stored at: {}".format(GDRIVE_DATASET_LINKS_FILE))

### **Read both dataset files (nodes and links) into two Spark Dataframes**

In [0]:
nodes_df = spark.read.load(GDRIVE_DATASET_NODES_FILE, 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                         )

In [0]:
links_df = spark.read.load(GDRIVE_DATASET_LINKS_FILE, 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                         )

#**2. Construct the `GraphFrames` graph object**

In [0]:
web_graph = GraphFrame(nodes_df, links_df) # Create the GraphFrame object from the 2 DataFrames

In [0]:
## Take a look at the DataFrames
web_graph.vertices.show(5, truncate=False)
web_graph.edges.show(5, truncate=False)

In [0]:
## Check the number of edges of each vertex
web_graph.degrees.show(10)

#**3. Execute PageRank on this graph**

The `GraphFrames` package has a nice and easy-to-use API for executing PageRank over a graph. There is a method called `pageRank` whose parameters are:
-  `resetProbability` which is the complement of the damping factor $d$ (i.e., $1-d$);
-  `tol` is the tolerance threshold used to determine the convergence of the PageRank vector;
-  `maxIter` specifies the total number of maximum iterations to be run (alternative to `tol`).


In [0]:
pr = web_graph.pageRank(resetProbability=0.15, tol=0.01)

In [0]:
# Look at the pagerank score for every vertex
pr.vertices.show(10)

In [0]:
# Sorting nodes by their value of PageRank (from the highest to the lowest)
pr.vertices.sort(['pagerank'], ascending=[0]).show(10)

In [0]:
# Scaling PageRank values so that PageRank vector entries sum up to 1
pr_sum = pr.vertices.groupBy().sum().collect()[0][0]

pr_norm = pr.vertices.withColumn("pagerank_norm", pr.vertices.pagerank/pr_sum)

In [0]:
pr_norm.sort(['pagerank'], ascending=[0]).show(10, truncate=False)

#**`GraphFrames` supports many other graph-based algorithms**

In addition to PageRank, `GraphFrames` provides support for the following graph-based algorithms:
-  Breadth-first search (BFS)
-  Connected components
-  Strongly connected components
-  Label Propagation Algorithm (LPA)
-  Shortest paths
-  Triangle count

For any further information, please refer to the [online user guide](https://graphframes.github.io/graphframes/docs/_site/user-guide.html).